In [ ]:
from analysis.paper import *

In [ ]:
ags = data.load()
ags = data.with_times(ags)

In [ ]:
plot_frontiers(ags)

In [ ]:
upload(plot_resid_var, ags)

In [ ]:
overleaf.table(hyperparams_table(), 'hyperparams')

In [ ]:
model.summary2()

In [ ]:
plot_resid_var(ags)

In [ ]:
upload(plot_resid_var, ags)